<a href="https://colab.research.google.com/github/xiu0327/lab/blob/main/(%EC%8B%A4%EC%8A%B51)CNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1) 데이터 셋 불러오기

In [ ]:
pip install konlpy

In [ ]:
import pandas as pd
train_data=pd.read_csv('/content/train_data.csv', encoding='utf-8')
test_data = pd.read_csv('/content/test_data.csv', encoding='utf-8')

In [ ]:
train_data

In [ ]:
import numpy as np
from collections import Counter
from konlpy.tag import Okt

# 데이터 중복 체크
train_data['text'].nunique(), train_data['label'].nunique()

# 데이터 중복 제거
train_data.drop_duplicates(subset='text', inplace=True)
test_data.drop_duplicates(subset='text', inplace=True)

# null값 유무
print(train_data.isnull().values.any())
print(test_data.isnull().values.any())

# 데이터 전처리
train_data['text'] = train_data['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data['text'].replace('', np.nan, inplace=True)
print(train_data.isnull().sum())

train_data = train_data.dropna(how='any') # null 값 제거

print(train_data.isnull().sum())

# 불용어 제거
from tqdm import tqdm
okt = Okt()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
X_train = []
for sentence in tqdm(train_data['text']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 정수 인코딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# 단어 집합이 생기는 동시에 각 단어에 고유한 정수가 부여됨. 등장횟수가 1회인 단어들은 배제.
threshold = 2
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

2) 단어 임베딩

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(vocab_size, oov_token='OOV')
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)

In [ ]:
# 패딩 작업
print('리뷰의 최대 길이 :',max(len(l) for l in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))

In [ ]:
def below_threshold_len(max_len, nested_list):
    count = 0
    for sentence in nested_list:
        if(len(sentence) <= max_len):
            count = count+1
    print("전체 샘플 중 길이가 "+str(max_len)+" 이하인 샘플의 비율 : {}".format((count / len(nested_list))*100))

max_len = 100
below_threshold_len(max_len, X_train)

3) 패딩

In [ ]:
X_train = pad_sequences(X_train, maxlen=max_len)

4) 모델 설정

In [ ]:
import re
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint  

model = Sequential()
embedding_dim = 100
hidden_units= 128
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

embedding_dim = 256
dropout_ratio = 0.3
num_filters = 256
kernel_size = 3
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dropout(dropout_ratio))
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_units, activation='relu'))
model.add(Dropout(dropout_ratio))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

embedding_dim = 256
dropout_ratio = 0.3
num_filters = 256
kernel_size = 3
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dropout(dropout_ratio))
model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(LSTM(hidden_units))
model.add(Dropout(dropout_ratio))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

y_train = train_data['label'].values

5) 모델 훈련

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history=model.fit(X_train, y_train, epochs=1, callbacks=[es,mc], batch_size=256, validation_split=0.2)

In [ ]:
test_data = pd.read_csv('/content/test_data.csv', encoding='utf-8')
print("전체 테스트 데이터 개수 : {}".format(len(test_data)))

test_data['text'].nunique()

test_data.drop_duplicates(subset=['text'], inplace=True)
print("중복을 제거한 테스트 데이터 개수 : {}".format(len(test_data)))

print(test_data.isnull().values.any())

test_data['text'] = test_data['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_data['text'].replace('', np.nan, inplace=True)
test_data = test_data.dropna(how='any')
print("전처리 후 테스트 데이터 개수 : {}".format(len(test_data)))

X_test = []
for sentence in tqdm(test_data['text']):
    tokenized_sentence = okt.morphs(sentence, stem=True)
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords]
    X_test.append(stopwords_removed_sentence)

# test 데이터 정수 인코딩
X_test = tokenizer.texts_to_sequences(X_test)

print(X_test)

X_test = pad_sequences(X_test, maxlen = max_len)
y_test = test_data['label'].values



In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))